In [1]:
import boto3
import pandas as pd
import numpy as np

In [2]:
def list_bucket_contents(bucket, match=''):
    s3_resource = boto3.resource('s3')
    bucket_resource = s3_resource.Bucket(bucket)
    for key in bucket_resource.objects.all():
        if match in key.key:
            print(key.key)

In [3]:
list_bucket_contents(bucket='nyc-tlc', match='2020')

trip data/fhv_tripdata_2020-01.csv
trip data/fhv_tripdata_2020-02.csv
trip data/fhv_tripdata_2020-03.csv
trip data/fhv_tripdata_2020-04.csv
trip data/fhv_tripdata_2020-05.csv
trip data/fhv_tripdata_2020-06.csv
trip data/fhv_tripdata_2020-07.csv
trip data/fhv_tripdata_2020-08.csv
trip data/fhv_tripdata_2020-09.csv
trip data/fhv_tripdata_2020-10.csv
trip data/fhv_tripdata_2020-11.csv
trip data/fhv_tripdata_2020-12.csv
trip data/fhvhv_tripdata_2020-01.csv
trip data/fhvhv_tripdata_2020-02.csv
trip data/fhvhv_tripdata_2020-03.csv
trip data/fhvhv_tripdata_2020-04.csv
trip data/fhvhv_tripdata_2020-05.csv
trip data/fhvhv_tripdata_2020-06.csv
trip data/fhvhv_tripdata_2020-07.csv
trip data/fhvhv_tripdata_2020-08.csv
trip data/fhvhv_tripdata_2020-09.csv
trip data/fhvhv_tripdata_2020-10.csv
trip data/fhvhv_tripdata_2020-11.csv
trip data/fhvhv_tripdata_2020-12.csv
trip data/green_tripdata_2020-01.csv
trip data/green_tripdata_2020-02.csv
trip data/green_tripdata_2020-03.csv
trip data/green_tripdata_

In [68]:
df = pd.read_csv('s3://nyc-tlc/trip data/yellow_tripdata_2020-04.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [69]:
df.head()
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge'],
      dtype='object')

In [70]:
df.to_csv('../data/raw/yellow_tripdata_2020-04.csv')

In [72]:
df_cleaned = df.copy()
df_cleaned.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge'],
      dtype='object')

In [73]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [75]:
from src.features.dates import convert_to_date

In [76]:
df_cleaned = convert_to_date(df_cleaned, ['tpep_pickup_datetime', 'tpep_dropoff_datetime'])

In [80]:
df_cleaned['trip_duration'] = (df_cleaned['tpep_dropoff_datetime'] - df_cleaned['tpep_pickup_datetime']).dt.total_seconds()

0         1231.0
1          805.0
2          539.0
3          600.0
4          302.0
           ...  
237988    1200.0
237989     300.0
237990     333.0
237991    1296.0
237992    1033.0
Length: 237993, dtype: float64


In [81]:
df_cleaned['trip_duration'] = pd.cut(df_cleaned['trip_duration'], bins=[0, 300, 600, 1800, 100000], labels=['x<5min', 'x<10min', 'x<30min', 'x>=30min'])

0         x<30min
1         x<30min
2         x<10min
3         x<10min
4         x<10min
           ...   
237988    x<30min
237989     x<5min
237990    x<10min
237991    x<30min
237992    x<30min
Name: trip_duration, Length: 237993, dtype: category
Categories (4, object): [x<5min < x<10min < x<30min < x>=30min]


In [14]:
df_cleaned['tpep_pickup_dayofmonth'] = df_cleaned['tpep_pickup_datetime'].dt.day

In [15]:
df_cleaned['tpep_pickup_hourofday'] = df_cleaned['tpep_pickup_datetime'].dt.hour

In [16]:
df_cleaned['tpep_pickup_dayofweek'] = df_cleaned['tpep_pickup_datetime'].dt.dayofweek

In [17]:
df_cleaned = pd.get_dummies(df_cleaned, columns=['VendorID', 'RatecodeID', 'store_and_fwd_flag'])

In [18]:
df_cleaned.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'PULocationID', 'DOLocationID'], axis=1, inplace=True)

In [19]:
df_cleaned.to_csv('../data/interim/yellow_tripdata_2020-04_prepared.csv')

In [107]:
from src.data.sets import split_sets_random

X_train, y_train, X_val, y_val, X_test, y_test = split_sets_random(df_cleaned, target_col='trip_duration', test_ratio=0.2, to_numpy=True)

[[2.0 Timestamp('2020-04-07 07:38:25') Timestamp('2020-04-07 07:47:00')
  ... 0.3 12.88 0.0]
 [1.0 Timestamp('2020-04-14 11:13:36') Timestamp('2020-04-14 11:14:53')
  ... 0.3 6.8 2.5]
 [1.0 Timestamp('2020-04-03 12:13:39') Timestamp('2020-04-03 12:24:32')
  ... 0.3 10.8 0.0]
 ...
 [1.0 Timestamp('2020-04-17 00:21:47') Timestamp('2020-04-17 00:46:41')
  ... 0.3 76.42 0.0]
 [nan Timestamp('2020-04-22 15:02:22') Timestamp('2020-04-22 15:25:28')
  ... 0.3 18.45 0.0]
 [1.0 Timestamp('2020-04-03 10:37:04') Timestamp('2020-04-03 10:41:36')
  ... 0.3 7.8 2.5]]


In [48]:
from src.data.sets import save_sets

save_sets(X_train, y_train, X_val, y_val, X_test, y_test, path='../data/processed/')

In [51]:
from src.models.null import NullModel

In [52]:
base_model = NullModel(target_type="classification")

In [109]:
y_train = pd.DataFrame(y_train).dropna().to_numpy()
y_base = base_model.fit_predict(y_train)

In [111]:
from src.models.performance import print_class_perf
print_class_perf(y_preds=y_base.flatten(), y_actuals=y_train, set_name='Training', average='weighted')

y_actuals [['x<10min']
 ['x<5min']
 ['x<30min']
 ...
 ['x<30min']
 ['x<30min']
 ['x<5min']]
y_preds ['x<10min' 'x<10min' 'x<10min' ... 'x<10min' 'x<10min' 'x<10min']
Accuracy Training: 0.3458092019464038
F1 Training: 0.17771316168422369
